In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.python.ops.gen_data_flow_ops import barrier_ready_size_eager_fallback
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import layers

In [ ]:
def UNet(pretrained_weights = None,input_size = (128,128,3)):
    inp = Input(input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(3, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inp, outputs=[conv10])

    return model

In [ ]:
import numpy as np
from glob import glob
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from ultralytics import YOLO

def identity_loss(y_true, y_pred):
  loss = tf.reduce_mean(tf.abs(y_true - y_pred))
  return loss

seg_model = YOLO("/content/drive/MyDrive/save_your_biometric_informations/runs/segment/train5/weights/best.pt")

data_path = glob("/content/drive/MyDrive/save_your_biometric_informations/*/images/*.jpg")
# data_path = ['/content/drive/MyDrive/save_your_biometric_informations/runs/영석1.jpg']
batch_size = 32
model = UNet()
model.compile(optimizer=Adam(learning_rate=3e-4), loss='binary_crossentropy')
dataset = list()

for data in tqdm(data_path):
  image = np.array(Image.open(data))
  h, w, _ = image.shape
  results = seg_model.predict(data)

  for result in results:
    masks = result.masks
    boxes = result.boxes
  if masks is not None:
    for mask, box in zip(masks.data, boxes.data):
      mask = cv2.resize(np.array(mask.cpu()).astype(np.uint8), dsize=(w, h), interpolation=cv2.INTER_CUBIC)
      bbox = int(box[1]), int(box[3]), int(box[0]), int(box[2])
      mask = mask[bbox[0]:bbox[1], bbox[2]:bbox[3]]
      sliced_img = image[bbox[0]:bbox[1], bbox[2]:bbox[3]]

      masked_img1 = mask * sliced_img[:, :, 0]
      masked_img2 = mask * sliced_img[:, :, 1]
      masked_img3 = mask * sliced_img[:, :, 2]
      masked_img = cv2.merge((masked_img1, masked_img2, masked_img3))

      masked_img = cv2.resize(masked_img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
      masked_img = np.array(masked_img) / 255.0

      dataset.append(masked_img)

train_dataset = tf.data.Dataset.from_tensor_slices((dataset, dataset)).shuffle(buffer_size=1000).batch(batch_size)

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
model = UNet()
model.compile(optimizer=Adam(learning_rate=3e-4), loss='binary_crossentropy')

In [ ]:
model.fit(train_dataset, epochs=50, batch_size=batch_size, shuffle=True)

In [ ]:
# 이미지 변조 실험

seg_model = YOLO("/content/drive/MyDrive/save_your_biometric_informations/runs/segment/train5/weights/best.pt")

data = '/content/drive/MyDrive/save_your_biometric_informations/runs/영석1.jpg'
image = np.array(Image.open(data))
h, w, _ = image.shape
results = seg_model.predict(data)

for result in results:
  masks = result.masks
  boxes = result.boxes

if masks is not None:
  for mask, box in zip(masks.data, boxes.data):
    mask = cv2.resize(np.array(mask.cpu()).astype(np.uint8), dsize=(w, h), interpolation=cv2.INTER_CUBIC)
    bbox = int(box[1]), int(box[3]), int(box[0]), int(box[2])
    mask = mask[bbox[0]:bbox[1], bbox[2]:bbox[3]]
    sliced_img = image[bbox[0]:bbox[1], bbox[2]:bbox[3]]

    masked_img1 = mask * sliced_img[:, :, 0]
    masked_img2 = mask * sliced_img[:, :, 1]
    masked_img3 = mask * sliced_img[:, :, 2]
    masked_img = cv2.merge((masked_img1, masked_img2, masked_img3))

    masked_img = cv2.resize(masked_img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    masked_img = np.array(masked_img) / 255.0

    output = model.predict(masked_img.reshape(1, 128, 128, 3))
    output = cv2.resize(output[0], dsize=(bbox[3]-bbox[2], bbox[1]-bbox[0]), interpolation=cv2.INTER_CUBIC)

    sliced_img = image[bbox[0]:bbox[1], bbox[2]:bbox[3]]

    mask = mask ^ True
    masked_img1 = mask * sliced_img[:, :, 0]
    masked_img2 = mask * sliced_img[:, :, 1]
    masked_img3 = mask * sliced_img[:, :, 2]
    masked_img = cv2.merge((masked_img1, masked_img2, masked_img3))

    output = np.add(masked_img, (output * 255).astype(np.int8)).astype(np.uint8)
    image[bbox[0]:bbox[1], bbox[2]:bbox[3]] = output

  plt.imshow(image)
  plt.show()

In [ ]:
import pathlib

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_models_dir = pathlib.Path("./")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"unet_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)